In [3]:
import sys
import os
import pandas as pd
import yfinance
sys.path.append("../src")
import json
import yfinance as yf

from data import data
from constants import Timeframe, Asset
from datetime import datetime, timedelta
from pydantic import BaseModel, ValidationError, PositiveInt
from datetime import datetime
from datetime import timedelta
from backtesting.backtest_config import BacktestConfig
from pydantic import BaseModel
from api.coindesk_api import get_OHLC


pd.set_option('display.max_columns', None)

In [8]:
# df = get_OHLC(
#     datetime(2024,1,1)
# )

df = yf.download("TSLA", start="2025-07-24", end="2025-07-25", interval="1M")

/tmp/ipykernel_21779/2556922349.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("TSLA", start="2025-07-24", end="2025-07-25", interval="1M")
[*********************100%***********************]  1 of 1 completed


In [9]:
df

Price,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA
Datetime,,,,,
2025-07-24 09:30:00,308.725098,310.065002,306.690002,310.065002,11036146
2025-07-24 09:31:00,304.480011,308.751801,304.309998,308.725006,2138302
2025-07-24 09:32:00,303.744995,305.500000,302.899994,304.549896,2009724
2025-07-24 09:33:00,302.714996,304.730011,301.000000,303.774414,2375019
2025-07-24 09:34:00,304.930511,305.200012,302.500000,302.809998,2267183
...,...,...,...,...,...
2025-07-24 15:55:00,306.470001,307.059998,305.709991,305.744995,749245
2025-07-24 15:56:00,306.299988,306.540009,306.170013,306.475006,392819


In [ ]:
df = data.get_df(
    Asset.ADA_USD,
    Timeframe.H1,
    path_append='.',
    file_type='PARQUET'
)


In [ ]:
df_ta = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)

In [ ]:
df_ta

In [ ]:
from_date = datetime(2024, 1, 1)
to_date = datetime.now().replace(microsecond=0, second=0, minute=0)
tick_limit=2000

def fetch_data(from_dt: datetime, to_dt: datetime):
    # This is your second function that gets called for each chunk
    # Replace this with actual logic (e.g., API calls, DB queries, etc.)
    print(f"Fetching from {from_dt} to {to_dt}")
    return [(from_dt, to_dt)]  # Dummy return for example

def chunked_fetch(start: datetime, end: datetime):
    chunk_size = timedelta(hours=2000)
    results = []

    current = start
    while current < end:
        chunk_end = min(current + chunk_size, end)
        chunk_result = fetch_data(current, chunk_end)
        results.extend(chunk_result)
        current = chunk_end

    return results

result = chunked_fetch(start=from_date, end=to_date)

print(result)

In [ ]:
df = data.get_df(Asset.ADA_USD, Timeframe.H1, ".", 'PARQUET')
df.head(10)

In [ ]:
class Trade(BaseModel):
    # price of quote crypto
    open_price: float = 0.0
    # total value of quote crypto in base
    open_value: float = 0.0
    # trade size of base crypto/currency(USD)
    total_quantity: float = 0.0
    # price of quote crypto
    current_price: float = 0.0
    # profit/loss in base crypto/currency(USD)
    profit_loss: float = 0.0
    # profit/loss percentage
    profit_loss_pct: float = 0.0
    # value of quote crypto in base
    current_value: float = 0.0
    average_entry_price = 0.0
    averaging_orders_prices: list = []
    averaging_order_sizes: list = []
    # list of the averaging order quantities (order_size / price), how many of the quote did you get
    averaging_orders_qty: list = []
    averaging_orders_filled: int = 0
    last_order_price: float = 0.0

#bot config
base_order_size = 20
averaging_order_size = 40
max_averaging_orders = 6
take_profit = 1.5
price_deviation = 1.5
averaging_order_size_mul = 1.28
averaging_order_step_mul = 1.85

def calc_avgeraging_order_prices(initial_price:float, deviation:float, multiplier:float):
        prices = []
        cumulative_multiplier = 0.0

        for i in range(6):
            cumulative_multiplier += multiplier ** i
            percent_drop = deviation * cumulative_multiplier
            price = initial_price * (1 - percent_drop / 100)
            prices.append(round(price, 7))

        return prices

def calc_averaging_order_size(avg_order_size: float, multiplier: float):
     sizes = [
          round(avg_order_size * (multiplier ** i), 2)
          for i in range(6)
     ]
     return sizes

def calc_profit_losses(current_price: float, total_cost: float, total_qty: float):
    current_value = current_price * total_qty
    profit_loss = current_value - total_cost
    profit_loss_pct = (profit_loss / total_cost) * 100

    return round(profit_loss, 6), round(profit_loss_pct, 6)

def open_trade(index, row, df, trade):
    trade.total_quantity = base_order_size / float(row.open)
    trade.open_value = float(trade.total_quantity * float(row.open))
    trade.last_order_price = float(row.open)
    trade.averaging_orders_prices = calc_avgeraging_order_prices(
        initial_price=float(row.open),
        deviation=price_deviation,
        multiplier=averaging_order_step_mul
        )
    trade.averaging_order_sizes = calc_averaging_order_size(
         avg_order_size=averaging_order_size,
         multiplier=averaging_order_size_mul
    )
    
    df.loc[index, 'trade_action'] = 'opened_trade'
    df.loc[index, 'trade_value'] = trade.open_value
    df.loc[index + 1, 'trade_action'] = 'in_trade'
    df.loc[index, 'avgeraging_orders'] = ','.join([str(item) + ',' for item in trade.averaging_orders_prices])
    df.loc[index, 'averaging_orders_sizes'] = ','.join([str(item) + ',' for item in trade.averaging_order_sizes])
     

def run_backtest(df):
    df_copy = df.copy()
    df_copy['trade_action'] = 'no_trade'
    df_copy['trade_value'] = 'None'
    df_copy['trade_profit_loss'] = 0.0
    df_copy['trade_profit_loss_pct'] = 0.0

    trades: list[Trade] = []

    current_trade = Trade()

    total_profit = 0.0
    hit_take_profit = False

    #account info
    starting_account_balance = 500.0
    current_account_balance = 500.0

    for index, row in enumerate(df_copy.itertuples(), 0):
        if current_trade.profit_loss_pct > take_profit:
            total_profit += current_trade.profit_loss
            df_copy.loc[index, 'trade_action'] = 'closed_trade'
            df_copy.loc[index + 1, 'trade_action'] = 'no_trade'
            trades.append(current_trade)
            current_trade = Trade()
        elif row.trade_action == 'in_trade':
            # maintain trade
            avg_orders_filled = current_trade.averaging_orders_filled
            if float(row.low) < current_trade.averaging_orders_prices[avg_orders_filled] and avg_orders_filled < 5:
                 current_trade.last_order_price = current_trade.averaging_orders_prices[avg_orders_filled]
                 current_trade.total_quantity += (current_trade.averaging_order_sizes[avg_orders_filled] / float(row.low))
                 current_trade.open_value += current_trade.averaging_order_sizes[avg_orders_filled]

                 df_copy.loc[index, 'trade_action'] = f"avg. order {avg_orders_filled + 1} filled"
                 if (avg_orders_filled < 5): current_trade.averaging_orders_filled += 1
            current_trade.current_value = float(current_trade.total_quantity * float(row.open))
            
            # current_price_deviation = ((float(row.open) - current_trade.last_order_price) / current_trade.last_order_price) * 100

            profit_loss, profit_loss_pct = calc_profit_losses(
                 current_price=float(row.high), 
                 total_cost=current_trade.open_value, 
                 total_qty=current_trade.total_quantity
                 )
            current_trade.profit_loss = profit_loss
            current_trade.profit_loss_pct = profit_loss_pct

            df_copy.loc[index, 'trade_value'] = current_trade.current_value
            df_copy.loc[index, 'trade_profit_loss'] = current_trade.profit_loss
            df_copy.loc[index, 'trade_profit_loss_pct'] = current_trade.profit_loss_pct
            # df_copy.loc[index, 'current_trade'] = current_trade
            df_copy.loc[index + 1, 'trade_action'] = 'in_trade'

        elif row.trade_action == 'no_trade':
            # open trade
            open_trade(index, row, df_copy, current_trade)

    return df_copy, trades, total_profit

In [ ]:
df_tested, trades, profit = run_backtest(df)


In [ ]:
data.save_df(df_tested, "ADA_tested", ".", "CSV")
print(len(trades))
print(sum(trade.profit_loss for trade in trades))
print(trades)
print(profit)
df_tested.head(60)

In [ ]:
%%timeit
run_backtest(df)

# Testing Ways to Loop or Vectorize DF:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def get_data(size = 10_000):
    df = pd.DataFrame()
    df['age'] = np.random.randint(0, 100, size)
    df['time_in_bed'] = np.random.randint(0, 9, size)
    df['pct_sleeping'] = np.random.rand(size)
    df['favorite_food'] = np.random.choice(['pizza', 'taco', 'ice-cream'], size)
    df['hate_food'] = np.random.choice(['broccoli', 'candy corn', 'eggs'], size)
    return df

## The problem
Reqard calc:
- If they were in bed for more than 5 hours AND they were sleeping for more than 50% we give them their favorite food.
- Otherwise we give them their hate food.
- If they are over 90 years old give their favorite foor regardless

In [ ]:
def reward_calc(row):
    if row['age'] >= 90:
        return row['favorite_food']
    if (row['time_in_bed'] > 5) & (row['pct_sleeping'] > 0.5):
        return row['favorite_food']
    return row['hate_food']

### Level 1 - Loop (Slowest)

In [ ]:
df = get_data()

In [ ]:
%%timeit
for index, row in df.iterrows():
    df.loc[index, 'reward'] = reward_calc(row)

### Level 2 - Apply

In [ ]:
df = get_data()

In [ ]:
%%timeit
df['reward'] = df.apply(reward_calc, axis=1)

### Level 3 - Vectorized (Fastest)

In [ ]:
df = get_data()

In [ ]:
%%timeit
df['reward'] = df['hate_food']
df.loc[
    ((df['pct_sleeping'] > 0.5) & (df['time_in_bed'] > 5) | (df['age'] > 90)), 
    'reward'
    ] = df['favorite_food']